<a href="https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2008%20-%20Mastering_Advanced_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from llama_index import SimpleDirectoryReader

# Specify the path to your file
file_path = "C:/Users/Christian/Documents/Cand_merc/ragbook-notebooks/notebooks/LLM.txt"

# Load the specific file
documents = SimpleDirectoryReader(input_files=[file_path]).load_data()


In [4]:
from llama_index import ServiceContext

# initialize service context (set chunk size)
service_context = ServiceContext.from_defaults(chunk_size=512, chunk_overlap=64)
node_parser = service_context.node_parser

nodes = node_parser.get_nodes_from_documents(documents)

In [5]:
from llama_index.vector_stores import DeepLakeVectorStore

my_activeloop_org_id = "hellum"
my_activeloop_dataset_name = "LlamaIndex_paulgraham_essay"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

# Create an index over the documnts
vector_store = DeepLakeVectorStore(dataset_path=dataset_path, overwrite=False)

Your Deep Lake dataset has been successfully created!


In [6]:
from llama_index.storage.storage_context import StorageContext

storage_context = StorageContext.from_defaults(vector_store=vector_store)
storage_context.docstore.add_documents(nodes)

In [7]:
from llama_index import VectorStoreIndex

vector_index = VectorStoreIndex(nodes, storage_context=storage_context)

Uploading data to deeplake dataset.


100%|██████████| 41/41 [00:01<00:00, 32.16it/s]


Dataset(path='hub://hellum/LlamaIndex_paulgraham_essay', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
   text       text      (41, 1)      str     None   
 metadata     json      (41, 1)      str     None   
 embedding  embedding  (41, 1536)  float32   None   
    id        text      (41, 1)      str     None   


In [8]:
query_engine = vector_index.as_query_engine(streaming=True, similarity_top_k=10)

In [9]:
streaming_response = query_engine.query(
    "What does Paul Graham do?",
)
streaming_response.print_response_stream()

Paul Graham works on a variety of projects, including writing essays, working on startup funding programs like the Summer Founders Program, giving talks, developing new programming languages like Bel and Arc, and engaging in art classes.

# SubQuestion Query Engine

In [10]:
query_engine = vector_index.as_query_engine(similarity_top_k=10)

In [11]:
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="pg_essay",
            description="Paul Graham essay on What I Worked On",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    service_context=service_context,
    use_async=True,
)

In [ ]:
response = query_engine.query(
    "How was Paul Grahams life different before, during, and after YC?"
)

In [14]:
print( ">>> The final response:\n", response )

NameError: name 'response' is not defined

# Cohere Rerank

In [17]:
import os
import cohere

# Get your cohere API key on: www.cohere.com
co = cohere.Client(os.environ['COHERE_API_KEY'])

# Example query and passages
query = "What is the capital of the United States?"
documents = [
   "Carson City is the capital city of the American state of Nevada. At the  2010 United States Census, Carson City had a population of 55,274.",
   "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean that are a political division controlled by the United States. Its capital is Saipan.",
   "Charlotte Amalie is the capital and largest city of the United States Virgin Islands. It has about 20,000 people. The city is on the island of Saint Thomas.",
   "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. ",
   "Capital punishment (the death penalty) has existed in the United States since before the United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.",
   "North Dakota is a state in the United States. 672,591 people lived in North Dakota in the year 2010. The capital and seat of government is Bismarck."
   ]

In [18]:
results = co.rerank(query=query, documents=documents, top_n=3, model='rerank-english-v2.0') # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.

for idx, r in enumerate(results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
  print(f"Document: {r.document['text']}")
  print(f"Relevance Score: {r.relevance_score:.2f}")
  print("\n")

Document Rank: 1, Document Index: 3
Document: Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. 
Relevance Score: 0.98


Document Rank: 2, Document Index: 1
Document: The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean that are a political division controlled by the United States. Its capital is Saipan.
Relevance Score: 0.30


Document Rank: 3, Document Index: 4
Document: Capital punishment (the death penalty) has existed in the United States since before the United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states.
Relevance Score: 0.28




# Cohere in LlamaIndex

In [19]:
import os
from llama_index.postprocessor.cohere_rerank import CohereRerank

cohere_rerank = CohereRerank(api_key=os.environ['COHERE_API_KEY'], top_n=2)

In [20]:
query_engine = vector_index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[cohere_rerank],
)

In [21]:
response = query_engine.query(
    "What did Sam Altman do in this essay?",
)
print( response )

Sam Altman was one of the individuals who were part of the first batch selected to be funded in the Summer Founders Program.
